### **Zadanie 1 úloha 2**

__Lukáš Bugaj__


__Adrián Maslák__

50/50

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from scipy.stats import iqr
from scipy import stats
import re

### Import datasetov

In [59]:
observation_df = pd.read_csv("069/observation.csv", sep='\t')
station_df = pd.read_csv("069/station.csv", sep='\t')
patient_df = pd.read_csv("069/patient.csv",sep = '\t', engine = "python")

## 1.2-A identifikácia a počiatočné riešenie problémov s dátami

V tomto kroku budeme analyzovať datasety a identifikovať jednotlivé problémy s dátami. Budeme sa pozerať na chýbajúce hodnoty, hodnoty v zlom formáte a podobne. Na začiatok si špecifikujeme niektoré veci:

##### 1. Dátumy budeme očakávať vo formáte mm/dd/yyyy
##### 2.  Súradnice budeme očakávať v rozmedzí latitude <-90, 90> a longtitude <-180, 180>

Hneď pri pohľade na výpis hlavy datasetu si všimame že v stĺpci current_location je nesprávny formát dát.


In [55]:
patient_df.head()

,address,job,ssn,name,company,blood_group,mail,username,registration,residence,current_location,user_id,station_ID
0,"Vale Carvalho, 75\nGuaratã\n24749298 da Rocha ...",Serralheiro,91728604303,Samuel Mendes,Garcia,A+,sophie45@uol.com.br,olivia61,"07/02/2022, 00:00:00",NaN,"(Decimal('-9.834721'), Decimal('-166.061899'))",221,438
1,"6808 Gary Lodge\nRobertfort, AZ 39881",NaN,157-43-4132,Philip Lee,Castro-Nielsen,A-,nathanhoffman@gmail.com,nicholas13,2021-03-10,NaN,"(Decimal('-58.933888'), Decimal('16.122641'))",993,138
2,Drewesring 6-2\n31689 Gotha,NaN,439-87-8316,Ercan Kreusel B.A.,Bender Kruschwitz KG,A-,lia29@web.de,ludgerpergande,2021-12-15,NaN,"(Decimal('-23.3593505'), Decimal('-103.776206'))",349,562
3,NaN,NaN,BGOTLL14R61D690T,Fabia Vento,Prodi e figli,A+,ottavio30@yahoo.com,enricotognazzi,"06/06/2020, 00:00:00",NaN,"(Decimal('72.988397'), Decimal('-123.076931'))",461,541
4,"Vicolo Morandini, 44 Piano 8\n27027, Gropello ...",NaN,SCDLSE60D12F851B,Antonino Cavanna,Lattuada Group,A-,romana82@gmail.com,borsierecarla,03 Nov 2022,NaN,"(Decimal('-60.724837'), Decimal('-65.915593'))",1405,738


Tu je vidieť, že namiesto štandardnej polohy tu sú dáta v tvare (Decimal('-60.724837'), Decimal('-65.915593')), taktiež nieje úplne zrejmé ktorá je latitude a longtitude hneď na prvý pohľad. Ďalej sa pozrieme na dátumy a chýbajúce hodnoty a skontrolujeme aj hodnoty súradníc v station.csv.

In [15]:

def parse_date(date_str):
    try:
        if isinstance(date_str, str):
            # Kontrola formátu mm/dd/yyyy
            if re.match(r'^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/(\d{4})$', date_str):
                return pd.to_datetime(date_str, format='%m/%d/%Y', errors='coerce')
        return pd.NaT
    except:
        return pd.NaT

def identify_data_issues(df, dataset_name):
    print(f"\n### Analýza problémov v datasete: {dataset_name} ###")

    # Kontrola chybějících hodnot
    missing_values = df.isnull().sum()
    print("\nChýbajúce hodnoty:")
    print(missing_values[missing_values > 0])

    # Identifikace sloupců s daty, které chceme ignorovat při detekci duplicit
    date_columns = []
    if dataset_name == "patient.csv" and 'registration' in df.columns:
        date_columns.append('registration')
    if dataset_name == "observation.csv" and 'observation_date' in df.columns:
        date_columns.append('observation_date')
    if dataset_name == "station.csv" and 'revision' in df.columns:
        date_columns.append('revision')

    non_date_columns = [col for col in df.columns if col not in date_columns]
    duplicates = df[non_date_columns].duplicated().sum() if non_date_columns else 0
    print(f"\nPočet duplicitných záznamov (ignorujúc dátumové stĺpce): {duplicates}")
    if duplicates > 0:
        print("Príklady duplicitných záznamov:")
        print(df[non_date_columns][df[non_date_columns].duplicated()].head())

    # Kontrola datumů pro konkrétní datasety
    if dataset_name in ["patient.csv", "observation.csv"]:
        for col in date_columns:
            df[col] = df[col].apply(parse_date)
            invalid_dates = df[col].isnull().sum()
            print(f"\nPočet neplatných dátumov v stĺpci '{col}' (očakávaný formát mm/dd/yyyy): {invalid_dates}")

    # Kontrola 'revision' pro station.csv
    if dataset_name == "station.csv" and 'revision' in df.columns:
        df['revision'] = df['revision'].apply(parse_date)
        inconsistent_dates = df['revision'].isnull().sum()
        print(f"\nPočet neplatných dátumov v stĺpci 'revision' (očakávaný formát mm/dd/yyyy: {inconsistent_dates}")

    # Kontrola formátu souřadnic
    if 'current_location' in df.columns:
        invalid_coords = df['current_location'].str.contains(r'[^\d\.\-\(\), ]', na=True).sum()
        print(f"\nPočet neplatných formátov súradníc: {invalid_coords}")

    # Kontrola platnosti zeměpisné šířky a délky
    if 'latitude' in df.columns and 'longitude' in df.columns:
        invalid_lat = df[(df['latitude'] < -90) | (df['latitude'] > 90)]['latitude'].count()
        invalid_lon = df[(df['longitude'] < -180) | (df['longitude'] > 180)]['longitude'].count()
        print(f"\nNeplatné zemepisné šírky: {invalid_lat}")
        print(f"Neplatné zemepisné dĺžky: {invalid_lon}")

print("=== Analýza datasetov ===")
identify_data_issues(station_df, "station.csv")
identify_data_issues(patient_df, "patient.csv")
identify_data_issues(observation_df, "observation.csv")

=== Analýza datasetov ===

### Analýza problémov v datasete: station.csv ###

Chýbajúce hodnoty:
code    3
dtype: int64

Počet duplicitných záznamov (ignorujúc dátumové stĺpce): 269
Príklady duplicitných záznamov:
     longitude           location              station code  latitude
80   -73.96250   America/New_York  Morningside Heights   US  40.81000
89   -40.74750  America/Fortaleza  Guaraciaba do Norte   BR  -4.16694
94    79.88300       Asia/Colombo              Hendala   LK   6.99090
112   51.34520      Europe/Samara             Otradnyy   RU  53.37596
119   73.97117       Asia/Karachi     Naushahra Virkan   PK  31.96258

Počet neplatných dátumov v stĺpci 'revision' (očakávaný formát mm/dd/yyyy: 836

Neplatné zemepisné šírky: 0
Neplatné zemepisné dĺžky: 0

### Analýza problémov v datasete: patient.csv ###

Chýbajúce hodnoty:
address              310
job                 1445
residence           2064
current_location     103
dtype: int64

Počet duplicitných záznamov (ignorujúc dátum

Po skontrolovaní datasetov máme odhalené viaceré nedokonalosti, ako napríklad 3 chýbajúce hodnoty code v station.csv. V station.csv máme taktiež duplicity čo sa týka názvov staníc ale to nás nezaujíma, pretože stanica si robí revíziu viac krát v priebehu jej fungovania.

Ďalej čo sa týka datasetu patient.csv tam sme si už všimli nesprávny formát súradníc, ale chýbajú nám tam aj viaceré chýbajúce hodnoty, ako napríklad:
address              310
job                 1445
residence           2064
current_location     103

Taktiež nami stanovené kritériá na formát dátumu tento dataset nespĺňa ani v jednom riadku.
Počet neplatných dátumov v stĺpci 'registration' (očakávaný formát mm/dd/yyyy): 2064.

Pohľadom na analýzu datasetu station.csv nieje v tomto datasete žiadna chyba spôsobená zberom dát, čiže nemá žiadne chýbajúce hodnoty ani zlý formát.

In [60]:
def parse_date(date_str):
    if not isinstance(date_str, str):
        return pd.NaT

    if ',' in date_str:
        date_str = date_str.split(',')[0].strip()

    # Zoznam podporovaných formátov
    formats = [
        '%Y-%m-%d',      # YYYY-MM-DD (napr. 2021-03-10)
        '%Y/%m/%d',      # YYYY/MM/DD (napr. 2021/08/02)
        '%m/%d/%Y',      # MM/DD/YYYY (napr. 07/02/2022)
        '%d %b %Y',      # DD Mon YYYY (napr. 03 Nov 2022)
        '%d %B %Y'       # DD Month YYYY (napr. 03 August 2022)
    ]
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
station_df['revision'] = station_df['revision'].apply(parse_date).dt.strftime('%m/%d/%Y')
patient_df['registration'] = patient_df['registration'].apply(parse_date).dt.strftime('%m/%d/%Y')
station_df.to_csv('069/station.csv', index=False, sep='\t')
patient_df.to_csv('069/patient.csv', index=False, sep='\t')

Týmto skriptom sa nám úspešne podarilo eliminovať rôzne formáty dátumov v súbore.

In [62]:
station_df.head()

,longitude,revision,location,station,code,latitude
0,20.89389,06/28/2023,Europe/Belgrade,Glogovac,XK,42.62833
1,77.24779,03/17/2025,Asia/Kolkata,Udumalaippettai,IN,10.58806
2,-6.00137,11/25/2023,Africa/Casablanca,Oulmes,MA,33.42585
3,120.56528,07/24/2023,Asia/Manila,Pilar,PH,14.66000
4,73.97117,11/17/2023,Asia/Karachi,Naushahra Virkan,PK,31.96258


In [63]:
patient_df.head()

,address,job,ssn,name,company,blood_group,mail,username,registration,residence,current_location,user_id,station_ID
0,"Vale Carvalho, 75\nGuaratã\n24749298 da Rocha ...",Serralheiro,91728604303,Samuel Mendes,Garcia,A+,sophie45@uol.com.br,olivia61,07/02/2022,NaN,"(Decimal('-9.834721'), Decimal('-166.061899'))",221,438
1,"6808 Gary Lodge\nRobertfort, AZ 39881",NaN,157-43-4132,Philip Lee,Castro-Nielsen,A-,nathanhoffman@gmail.com,nicholas13,03/10/2021,NaN,"(Decimal('-58.933888'), Decimal('16.122641'))",993,138
2,Drewesring 6-2\n31689 Gotha,NaN,439-87-8316,Ercan Kreusel B.A.,Bender Kruschwitz KG,A-,lia29@web.de,ludgerpergande,12/15/2021,NaN,"(Decimal('-23.3593505'), Decimal('-103.776206'))",349,562
3,NaN,NaN,BGOTLL14R61D690T,Fabia Vento,Prodi e figli,A+,ottavio30@yahoo.com,enricotognazzi,06/06/2020,NaN,"(Decimal('72.988397'), Decimal('-123.076931'))",461,541
4,"Vicolo Morandini, 44 Piano 8\n27027, Gropello ...",NaN,SCDLSE60D12F851B,Antonino Cavanna,Lattuada Group,A-,romana82@gmail.com,borsierecarla,11/03/2022,NaN,"(Decimal('-60.724837'), Decimal('-65.915593'))",1405,738


Ďalej sa pozrieme na zlú štruktúru dát v stĺpci current_location

In [65]:
def parse_coordinates(coord_str):
    if not isinstance(coord_str, str):
        return None, None
    pattern = r"Decimal\('([-\d.]+)'\)"
    matches = re.findall(pattern, coord_str)

    if len(matches) == 2:
        latitude = float(matches[0])
        longitude = float(matches[1])
        return latitude, longitude

    return None, None
patient_df[['latitude', 'longitude']] = patient_df['current_location'].apply(
    lambda x: pd.Series(parse_coordinates(x))
)
patient_df = patient_df.drop('current_location', axis=1)

patient_df.to_csv('069/patient.csv', index=False, sep='\t')

Tento skript úspešne vyparsuje nevhodnú štruktúru dát v stĺpci current location ktorý vyhodí, a pridá nové stĺpce latitude a longtitude.

In [66]:
patient_df.head()

,address,job,ssn,name,company,blood_group,mail,username,registration,residence,user_id,station_ID,latitude,longitude
0,"Vale Carvalho, 75\nGuaratã\n24749298 da Rocha ...",Serralheiro,91728604303,Samuel Mendes,Garcia,A+,sophie45@uol.com.br,olivia61,07/02/2022,NaN,221,438,-9.834721,-166.061899
1,"6808 Gary Lodge\nRobertfort, AZ 39881",NaN,157-43-4132,Philip Lee,Castro-Nielsen,A-,nathanhoffman@gmail.com,nicholas13,03/10/2021,NaN,993,138,-58.933888,16.122641
2,Drewesring 6-2\n31689 Gotha,NaN,439-87-8316,Ercan Kreusel B.A.,Bender Kruschwitz KG,A-,lia29@web.de,ludgerpergande,12/15/2021,NaN,349,562,-23.359351,-103.776206
3,NaN,NaN,BGOTLL14R61D690T,Fabia Vento,Prodi e figli,A+,ottavio30@yahoo.com,enricotognazzi,06/06/2020,NaN,461,541,72.988397,-123.076931
4,"Vicolo Morandini, 44 Piano 8\n27027, Gropello ...",NaN,SCDLSE60D12F851B,Antonino Cavanna,Lattuada Group,A-,romana82@gmail.com,borsierecarla,11/03/2022,NaN,1405,738,-60.724837,-65.915593


In [11]:
df = observation_df
column = 'HR'
print(f"\n### Detekcia a spracovanie vychýlených hodnôt v stĺpci {column} ###")

    # Technika 1: IQR (odstránenie vychýlených hodnôt)
Q1 = df[column].quantile(0.25)
Q3 = df[column].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers_iqr = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
print(f"IQR - Počet vychýlených hodnôt: {len(outliers_iqr)}")
df_no_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    # Technika 2: Z-skóre (nahradenie hraničnými hodnotami 5% a 95%)
z_scores = np.abs(stats.zscore(df[column].dropna()))
threshold = 3
outliers_z = df[z_scores > threshold][column]
print(f"Z-skóre - Počet vychýlených hodnôt: {len(outliers_z)}")
lower_percentile = df[column].quantile(0.05)
upper_percentile = df[column].quantile(0.95)
df_replaced = df.copy()
df_replaced.loc[z_scores > threshold, column] = df_replaced[column].clip(lower=lower_percentile, upper=upper_percentile)



### Detekcia a spracovanie vychýlených hodnôt v stĺpci HR ###
IQR - Počet vychýlených hodnôt: 80
Z-skóre - Počet vychýlených hodnôt: 29
